In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib
import random
import pandas as pd
import sys

from credentials import CONNECTION_INFO
from constants import *

import encoders
import db_connect
import helpers

# Callbacks for use with TensorFlow
from tf_helpers import modulo_output, modulo_distance_loss, modulo_distance_accuracy, modulo_rounded_accuracy, initialize_save_best

2024-11-30 20:28:04.677164: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 20:28:04.691641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 20:28:04.705906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 20:28:04.709548: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 20:28:04.720988: I tensorflow/core/platform/cpu_feature_guar

## Config
This notebook has a lot of options to adjust, most of which are controlled here.

In [2]:
ENCODER = encoders.ENCODER_CAESAR
CHUNK_SIZE = 128 # Was 512
PROCESSING_UNITS = CHUNK_SIZE // 4

EXTRA_CHECKS = True # Whether to run some (potentially slow) debug checks

INFER_TEXT = False
INFER_KEY = not INFER_TEXT

USE_CUSTOM_METRICS = True
USE_CUSTOM_OUTPUT_ACTIVATION = True

if INFER_TEXT:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mean_squared_error"
    OUTPUT_SIZE = CHUNK_SIZE
    OPTIMIZER = "sgd"
else:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mae"
    OPTIMIZER = "adamax"    

    if ENCODER == encoders.ENCODER_CAESAR:
        OUTPUT_SIZE = 1
    elif ENCODER == encoders.ENCODER_SUBST:
        OUTPUT_SIZE = len(encoders.CHARSET)
    else:
        raise Exception(f"Unsupported encoder {ENCODER}")

ENCRYPTED_FILE_LIMIT = -1 # -1 to disable limit

BASE_TRAIN_PCT = 0.75   # Start here. If train or test count would exceed the max, reduce it. Note 0.75 is the default.
MAX_TRAIN_COUNT = 100000 # -1 to disable; some setups start running out of memory around 100K
MAX_TEST_COUNT =  100000 # -1 to disable
SPLIT_SEED = 42

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
BEST_PATH = './saved_models/best.keras'

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
TUNE_QUICKLY = False # Set True to sanity check the model builder
BUILD_NETWORK = not TUNE_NETWORK
TRAIN_MODEL = BUILD_NETWORK

TUNER_DIRECTORY = "tuner_projects"
TUNER_PROJECT_NAME = "KT"

EPOCHS = 100
BATCH_SIZE = int(max(32, round(256 * (512/CHUNK_SIZE)))) # Default is 32 -- going higher speeds things up a LOT, but may cause memory problems
SCALE = True

CHUNK_SIZE, PROCESSING_UNITS, BATCH_SIZE, OUTPUT_SIZE

(128, 32, 1024, 1)

# Data Retrieval and Structuring

In [3]:
db = db_connect.DB(CONNECTION_INFO)

with db.get_session() as session:
    # Get database IDs for encoders and key types
    (encoder_ids, key_type_id) = db.get_id_maps(session)

    # Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
    (sid_to_p, sid_to_c) = db.get_source_maps(session, ENCRYPTED_FILE_LIMIT, encoder_ids[ENCODER], test_only=False)

len(sid_to_p), len(sid_to_c)

(35, 35)

In [4]:
# Build up the features (X, the cipher texts as offsets) and targets (y, either the plain texts as offsets OR the key).
# Note targets are not necessarily unique.
X = []
y = []

with db.get_session() as session:
    for sid in sid_to_p:
        if INFER_TEXT:
            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
            target_chunks = helpers.chunkify(plaintext, CHUNK_SIZE)    
    
        for c in sid_to_c[sid]:
            ciphertext = encoders.string_to_offsets(helpers.read_text_file(c.path))
            feature_chunks = helpers.chunkify(ciphertext, CHUNK_SIZE)

            if INFER_KEY:                
                if ENCODER == encoders.ENCODER_CAESAR:
                    key_value = float(db.get_key_by_id(session, c.key_id).value)

                    if EXTRA_CHECKS:
                        # Decode with the key we got from the DB, make sure it actually works
                        CHECK_CHANCE = 0.1
                        if random.random() < CHECK_CHANCE:
                            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
                            plainttext_str = encoders.offsets_to_string(plaintext)
                            ciphertext_str = encoders.offsets_to_string(ciphertext)
                            decoded_str = encoders.decode_caesar(ciphertext_str, int(key_value))
                            if decoded_str != plainttext_str:                                                    
                                print(decoded_str == plainttext_str)
                                print(decoded_str[0:128], plainttext_str[0:128])
                                raise Exception("Decode error")
                    
                elif ENCODER == encoders.ENCODER_SUBST:
                    key_str = db.get_key_by_id(session, c.key_id).value
                    key_value_ints = encoders.string_to_offsets(key_str)
                    key_value = np.array(key_value_ints).astype(float)

                else:
                    raise Exception(f"Unsupported encoder {ENCODER}")
        
            for i in range (len(feature_chunks)):
                X.append(np.array(feature_chunks[i]).astype(float))

                if INFER_TEXT:
                    y.append(np.array(target_chunks[i]).astype(float))

                if INFER_KEY:
                    y.append(key_value)

X = np.array(X)
y = np.array(y)

X.shape, y.shape, sys.getsizeof(X), sys.getsizeof(y)

((383568, 128), (383568,), 392773760, 3068656)

In [5]:
# Split the preprocessed data into a training and testing dataset
# Note we have excluded "test_only" files above, they will be used for later validation.

train_count = int(round(len(y) * BASE_TRAIN_PCT))
if train_count > MAX_TRAIN_COUNT and MAX_TRAIN_COUNT > -1:
    print(f"Train count would be {train_count}")
    train_count = int(MAX_TRAIN_COUNT)
print(f"Train count is {train_count}")

test_count = len(y) - train_count
if test_count > MAX_TEST_COUNT and MAX_TEST_COUNT > -1:
    print(f"Test count would be {test_count}")
    test_count = int(MAX_TEST_COUNT)
print(f"Test count is {test_count}")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_count, test_size=test_count, random_state=SPLIT_SEED)
print( "Initial counts: ", len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

# The pre-split data sets are no longer needed, and take up a lot of memory, so get rid of them
del X
del y

Train count would be 287676
Train count is 100000
Test count would be 283568
Test count is 100000
Initial counts:  383568 383568 100000 100000 100000 100000


In [6]:
if SCALE:
    # Create a StandardScaler instances
    scaler = StandardScaler()
    
    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)
    
    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)    
else:
    # Leave the inputs unscaled
    X_train_scaled = X_train
    X_test_scaled = X_test

to_show = min(16, CHUNK_SIZE)
X_train_scaled.shape, X_test_scaled.shape, X_train_scaled[0][0:to_show], X_test_scaled[0][0:to_show]

((100000, 128),
 (100000, 128),
 array([-1.62550526, -1.62427011, -0.20888303,  0.84840755, -1.62518328,
         1.5517651 , -0.56233216, -1.62134237,  1.55200644, -1.26825244,
        -0.56097728,  1.55136447, -1.61839792, -1.62033258, -1.27146587,
         1.5453671 ]),
 array([-0.21232657,  0.49642689,  0.14329365, -0.91822486,  0.4949298 ,
         1.19911045, -0.91467281,  0.14143082,  0.49329087, -1.62174667,
         0.85047274, -0.91853411,  1.20365493, -1.62033258,  0.49254304,
         0.13569825]))

# Hyperband Tuning

In [7]:
import model_tuner

MAX_EPOCHS_PER_MODEL = 20 # Meant to get a decent idea of parameter, not create a final model. Behaves oddly below 3.
HYPERBAND_ITERATIONS = 2  # "Number of times to iterate over the full Hyperband algorithm"
EXECUTIONS_PER_TRIAL = 2  # Training from scratch
SEARCH_FIT_EPOCHS = 20    # Epochs for each attempt to do a fit, I think. Not sure how this relates to MAX_EPOCHS_PER_MODEL.
OVERWRITE = True          # I'm hoping to be able to interrupt a run and resume it later

input_shape = (CHUNK_SIZE,1)
mr_t = model_tuner.ModelTuner(input_shape, OUTPUT_SIZE, CHUNK_SIZE, BATCH_SIZE)

# All-encompassing optimization parameter choices. Do not try to use all of them at once...
mr_t.CHOICES_PROCESSING_UNITS = [1, CHUNK_SIZE // 16, CHUNK_SIZE // 4, CHUNK_SIZE, CHUNK_SIZE * 2]
mr_t.CHOICES_ACTIVATIONS = ["elu", "gelu", "hard_sigmoid", "hard_silu", "hard_swish", "leaky_relu", "linear", "log_softmax", "mish",
        "relu", "relu6", "selu", "sigmoid", "silu", "softmax", "softplus", "softsign", "swish", "tanh"]
mr_t.CHOICES_FANCY_TOPO = ["GRU", "RNN", "LSTM", "GRU-RNN", "GRU-LSTM", "GRU-RNN-LSTM"]
mr_t.CHOICES_USE_OUTPUT_LIMITER = [True, False] # Prefers True
mr_t.CHOICES_OPTIMIZER = ["adamax", "sgd", "RMSProp"]

# Narrow down the choices as needed.
mr_t.CHOICES_PROCESSING_UNITS = [CHUNK_SIZE]
mr_t.CHOICES_FANCY_TOPO = ["NONE"]
mr_t.CHOICES_USE_OUTPUT_LIMITER = [True]
mr_t.CHOICES_OPTIMIZER = ["adamax"]

if TUNE_QUICKLY:
    MAX_EPOCHS_PER_MODEL = 3
    HYPERBAND_ITERATIONS = 1
    EXECUTIONS_PER_TRIAL = 1
    SEARCH_FIT_EPOCHS = 4

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    return mr_t.CreateModel(hp)

# Run the kerastuner search for best hyperparameters
if TUNE_NETWORK:
    if USE_CUSTOM_METRICS:
        objective = kt.Objective("val_modulo_distance_accuracy", direction="max")
    else:
        objective = kt.Objective(f"val_{MAIN_ACCURACY_METRIC}", direction="max")

    tuner = kt.Hyperband(
        create_model,
        objective=objective,
        max_epochs=MAX_EPOCHS_PER_MODEL,
        hyperband_iterations=HYPERBAND_ITERATIONS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        overwrite=OVERWRITE,
        directory=TUNER_DIRECTORY,
        project_name=TUNER_PROJECT_NAME)
    tuner.search(X_train_scaled, y_train, epochs=SEARCH_FIT_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test_scaled,y_test))
    
    best_hyper = tuner.get_best_hyperparameters(1)[0]
    print(f"Best Hyper Values: {best_hyper.values}")
    
    nn = tuner.get_best_models(1)[0]
    eval_results = nn.evaluate(X_test_scaled, y_test, verbose=2, batch_size=BATCH_SIZE )
    print(f"Best Model Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

    nn.save("./saved_models/tuned.keras")

# Model Reload /Creation

In [8]:
if BUILD_NETWORK:
    print("Building new model")
    nn = tf.keras.models.Sequential()

    NEW_ONE = True
    if NEW_ONE:

        if False:
            input_shape = (1,1, CHUNK_SIZE)
            foo = tf.keras.layers.Input(shape=input_shape[1:], name ="InputLayer")
            seq = tf.keras.layers.LSTM(units= 64, return_sequences = True, name='Layer1')(foo)  # output must be batchsize x timesteps x units
    
            plaintext_vocab_size = OUTPUT_MAX
            output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
            
            nn = tf.keras.Model(inputs = foo, outputs = output)

        if True:
            #input_shape = (CHUNK_SIZE,1)
            #input_shape = (None, CHUNK_SIZE, 1)
            input_shape = (None, 1, CHUNK_SIZE)
            input_layer = tf.keras.Input(shape=input_shape[1:], name="Input_Layer")
            nn.add(input_layer)
    
            activation_A = "tanh"
            recurrent_activation_A = "sigmoid"
            lstm_layer = tf.keras.layers.LSTM(PROCESSING_UNITS, return_sequences=True, activation=activation_A, recurrent_activation=recurrent_activation_A,
                name=f"A_LSTM_{activation_A}_{recurrent_activation_A}")
            nn.add(lstm_layer)
    
            # Rescale that 0-1 value from Sigmoid to the correct output range
            #nn.add(tf.keras.layers.Rescaling(scale=OUTPUT_MAX, offset=0, name="Rescaler"))
            #nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation=modulo_output, name="Output_Limiter"))
            #nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation="linear", name="Nothing"))

            #plaintext_vocab_size = OUTPUT_MAX
            #output_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units = plaintext_vocab_size, activation='softmax', name='TD_Layer'), input_shape=(1,2,3))
            #output_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units = OUTPUT_SIZE, activation=modulo_output, name='TD_Layer'), input_shape=(1, 1, PROCESSING_UNITS))
            output_layer = tf.keras.layers.Dense(units = OUTPUT_SIZE, activation=modulo_output, name='Modulo_Layer')
            nn.add(output_layer)

    else:
        # This was looking promising for a while...

        # Input layer
        nn.add(tf.keras.layers.Embedding(input_dim=CHUNK_SIZE, output_dim=PROCESSING_UNITS, name="Embedding_Input"))
    
        # This LSTM layer seems to do most of the real work
        nn.add(tf.keras.layers.LSTM(1, name="LSTM"))
    
        # Sigmoid layer produces an output between 0 and 1
        #nn.add(tf.keras.layers.Dense(units=PROCESSING_UNITS, activation="sigmoid", name="Sigmoid"))
    
        # Rescale that 0-1 value from Sigmoid to the correct output range
        nn.add(tf.keras.layers.Rescaling(scale=OUTPUT_MAX, offset=0, name="Rescaler")) # Input is 0-1
    
        # Do modulo division to enforce output range limit
        # Note this seems like it should be totally redundant. But the Tuner results suggest that including both
        # mechanisms (Sigmoid + Rescaling, and Modulo Division) produces better results. I don't know why.
        nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation=modulo_output, name="Output_Limiter"))

    # Check the structure of the model
    print(f"Input shape: {nn.input_shape}")
    print(nn.summary())

Building new model
Input shape: (None, 1, 128)


I0000 00:00:1733027296.853842   21131 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733027296.965129   21131 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733027296.965178   21131 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733027296.967766   21131 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733027296.967823   21131 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ A_LSTM_tanh_sigmoid (LSTM)      │ (None, 1, 32)          │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Modulo_Layer (Dense)            │ (None, 1, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,641 (80.63 KB)

 Trainable params: 20,641 (80.63 KB)

 Non-trainable params: 0 (0.00 B)

None


# Model Training

In [9]:
# Reset the "best" score.
# If you're manually training iteratively, comment this out to preserve the best-ness:
model_checkpoint_callback = initialize_save_best(BEST_PATH)

# It can be helpful to load the best, train some more, and try to improve it:
if LOAD_BEST_MODEL:
    if os.path.exists(BEST_PATH):
        print(f"Loading model from {BEST_PATH}")
        nn = tf.keras.models.load_model(BEST_PATH,
            custom_objects={
                'modulo_distance_loss': modulo_distance_loss,
                'modulo_distance_accuracy': modulo_distance_accuracy,
                'modulo_rounded_accuracy': modulo_rounded_accuracy,
                'modulo_output': modulo_output
        })

# Train the model
if TRAIN_MODEL:
    # Decide what metrics to use
    if USE_CUSTOM_METRICS:
        loss = modulo_distance_loss
        metrics = [modulo_distance_accuracy, modulo_rounded_accuracy]
    else:
        loss = LOSS_METRIC
        metrics = [MAIN_ACCURACY_METRIC]

    print(nn.summary())
    print(f"Training model")
    
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    else:
        callbacks = None
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss=loss, optimizer=OPTIMIZER, metrics=metrics)
    
    # Fit the model to the training data
    tmp_x = X_train_scaled.reshape((-1, CHUNK_SIZE, 1)) 
    tmp_y = y_train.reshape((-1, OUTPUT_SIZE, 1)) 
    print(X_train_scaled.shape, y_train.shape, tmp_x.shape, tmp_y.shape)
    #fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)
    fit_model = nn.fit(tmp_x, tmp_y, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)

nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ A_LSTM_tanh_sigmoid (LSTM)      │ (None, 1, 32)          │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Modulo_Layer (Dense)            │ (None, 1, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,641 (80.63 KB)

 Trainable params: 20,641 (80.63 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ A_LSTM_tanh_sigmoid (LSTM)      │ (None, 1, 32)          │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Modulo_Layer (Dense)            │ (None, 1, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,925 (241.90 KB)

 Trainable params: 20,641 (80.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 41,284 (161.27 KB)

In [10]:
# Evaluate the model using the test data

tmp_x = X_test_scaled.reshape((-1, CHUNK_SIZE, 1))
tmp_y = y_test.reshape((-1, OUTPUT_SIZE, 1)) 

if USE_CUSTOM_METRICS and INFER_KEY:    
    print("Evaluating with model.predict() ...")    
    raw_pred = nn.predict(tmp_x, batch_size=BATCH_SIZE)
    
    print("Shapes", tmp_x.shape, raw_pred.shape, y_test.shape)
    preds = np.array([np.mean(raw_pred[x,:,:]) for x in range(raw_pred.shape[0])]).astype(np.float64)

    # My custom loss and accuracy functions are running out of memory for some reason...
    if True:
        #y_pred = tf.constant(np.array(raw_pred).astype(np.float64))
        y_pred = preds
        loss = modulo_distance_loss(y_test, y_pred)
        accuracy_distance = modulo_distance_accuracy(y_test, y_pred)
        accuracy_rounded = modulo_rounded_accuracy(y_test, y_pred)
        print(f"Loss: {loss:0.6}, Accuracy (Distance): {accuracy_distance:0.6}, Accuracy (Rounded): {accuracy_rounded:0.6}")

    if INFER_KEY:
        #pred_pd = pd.DataFrame(raw_pred)
        pred_pd = pd.DataFrame(preds)
        true_pd = pd.DataFrame(y_test)
        print("Inferred key distribution:\n", pred_pd.describe())
        #print("True     key distribution:\n", true_pd.describe())

print("Evaluating with model.evaluate() ...")
print(X_train_scaled.shape, y_train.shape, X_test_scaled.shape, y_test.shape, tmp_x.shape, tmp_y.shape)
#eval_results = nn.evaluate(tmp_x, y_test, verbose=2, batch_size=BATCH_SIZE)
eval_results = nn.evaluate(tmp_x, tmp_y[:,0,0], verbose=2, batch_size=BATCH_SIZE)
print(f"With X_test_scaled, Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

Evaluating with model.predict() ...
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Shapes (100000, 128, 1) (100000, 128, 1) (100000,)
Loss: 0.847665, Accuracy (Distance): 0.830467, Accuracy (Rounded): 0.50466
Inferred key distribution:
                    0
count  100000.000000
mean        4.553149
std         1.925316
min         0.820821
25%         2.887615
50%         4.558221
75%         6.205571
max         8.670630
Evaluating with model.evaluate() ...
(100000, 128) (100000,) (100000, 128) (100000,) (100000, 128, 1) (100000, 1, 1)
98/98 - 7s - 73ms/step - loss: 2.4457 - modulo_distance_accuracy: 0.5109 - modulo_rounded_accuracy: 0.1094
With X_test_scaled, Loss: 2.44565749168396, Accuracy: [0.510863184928894, 0.10939842462539673]


# Model Usefulness Spot-Check

In [11]:
# These only really work for the Caesar Cipher

def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    offset_chunks = helpers.chunkify(encoders.string_to_offsets(ciphertext), CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(offset_chunks, model, scaler, input_already_scaled = False)
    rounded = np.rint(decoded_chunks.flatten()).astype(int)
    return encoders.offsets_to_string(rounded)

def infer_key_with_model(ciphertext: str, model, scaler) -> int:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)    
    scaled_chunks = scaler.transform(chunks)
    #keys = model.predict(scaled_chunk)
    tmp_x = np.array(scaled_chunks).reshape((-1, CHUNK_SIZE, 1))
    keys = model.predict(tmp_x)
    key = int(round(np.median(keys)))
    return key

if INFER_TEXT:    
    CHUNKS_TO_CHECK = 2
else:
    CHUNKS_TO_CHECK = 20

cipher_file_db = sid_to_c[list(sid_to_c.keys())[0]][0]
ciphertext_path = cipher_file_db.path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE * CHUNKS_TO_CHECK]

if INFER_TEXT:    
    print("Decoded   : ", decode_text_with_model(ciphertext, nn, X_scaler))
if INFER_KEY:
    with db.get_session() as session:
        correct_key = db.get_key_by_id(session, cipher_file_db.key_id).value
    print("Correct Key: ", correct_key)
    
    inferred_key = infer_key_with_model(ciphertext, nn, X_scaler)
    print("Inferred Key: ", inferred_key)
    
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    scaled_chunks = scaler.transform(chunks)
    pred = nn.predict(scaled_chunks)
    
    #tmp_x = np.array(scaled_chunks).reshape((-1, CHUNK_SIZE, 1))
    #pred = nn.predict(tmp_x)

    #print(pred)
    #print(pd.DataFrame(pred).describe())


Correct Key:  9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Inferred Key:  6


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None